In [7]:
import pandas as pd
import os
import numpy as np
import re

In [2]:
# get samples 
samples = pd.read_csv("../../config/samples.csv")["ID"]
samples

0               Adelpha_iphiclus
1     Anartia_jatrophae_saturata
2               Araschnia_levana
3                 Auzakia_danava
4                 Baeotus_beotus
5          Catacroptera_cloanthe
6                Chalinga_pratti
7       Diaethria_gabaza_eupepla
8            Doleschallia_melana
9                  Eurema_blanda
10           Hypolimnas_usambara
11               Junonia_villida
12             Kallima_paralekta
13             Kallimoides_rumia
14               Litinga_cottini
15              Mallika_jacksoni
16                Moduza_procris
17               Parasarpa_zayla
18            Phaedyma_columella
19                Precis_pelarga
20       Protogoniomorpha_temora
21                 Salamis_cacta
22             Smyrna_blomfildia
23                Tacola_larymna
24                   Yoma_algina
25         Tacola_larymna_subset
26            Yoma_algina_subset
Name: ID, dtype: object

In [3]:
seqkit_dir = "../../results/seqkit/"

# list to hold dataframes
list_seqkit = []
# iterate throught seqkit results files
for file in os.listdir(seqkit_dir):
    if file.endswith(".txt"):
        # use regex separator for variable whitespace with python engine
        df = pd.read_csv(os.path.join(seqkit_dir, file), sep=r"\s+")
        # append to list
        list_seqkit.append(df)
# concat list
dat_seqkit = pd.concat(list_seqkit, ignore_index=True)
# clean up dataframe
dat_seqkit.drop("format", axis=1, inplace=True)
dat_seqkit.drop("type", axis=1, inplace=True)
dat_seqkit["ID"] = dat_seqkit["file"].str.replace(".fasta", "", regex=False)
dat_seqkit.drop("file", axis=1, inplace=True)
dat_seqkit.rename(columns={"num_seqs": "N seqs", 
                           "sum_len": "Sum length", 
                           "min_len": "Min. length", 
                           "avg_len": "Avg. length", 
                           "max_len": "Max. length"}, inplace=True)
# join with sample names
dat_seqkit = pd.merge(samples, dat_seqkit, how="left", on="ID")
# add column to state if assembly present
dat_seqkit["Assembled sequence"] = dat_seqkit["N seqs"].apply(lambda x: "True" if x > 0 else "False")
dat_seqkit = dat_seqkit[["ID", "Assembled sequence", "N seqs", "Sum length", "Min. length", "Avg. length", "Max. length"]]
dat_seqkit

,ID,Assembled sequence,N seqs,Sum length,Min. length,Avg. length,Max. length
0,Adelpha_iphiclus,True,1.0,"15,263","15,263","15,263","15,263"
1,Anartia_jatrophae_saturata,True,1.0,"15,281","15,281","15,281","15,281"
2,Araschnia_levana,True,1.0,"15,207","15,207","15,207","15,207"
3,Auzakia_danava,True,3.0,"15,017","3,204","5,005.7","6,944"
4,Baeotus_beotus,True,1.0,"15,131","15,131","15,131","15,131"
5,Catacroptera_cloanthe,True,1.0,"13,058","13,058","13,058","13,058"
6,Chalinga_pratti,True,1.0,"15,244","15,244","15,244","15,244"
7,Diaethria_gabaza_eupepla,True,1.0,"15,156","15,156","15,156","15,156"
8,Doleschallia_melana,True,1.0,"15,379","15,379","15,379","15,379"
9,Eurema_blanda,True,1.0,"15,123","15,123","15,123","15,123"


In [4]:
blobtools_dir = "../../results/blobtools/"

# list to hold dataframes
list_blobtools = []
# iterate throught blobtools results files
for directory in os.listdir(blobtools_dir):
    if os.path.isdir(os.path.join(blobtools_dir, directory)):
        for file in os.listdir(os.path.join(blobtools_dir, directory)):
            if file == "table.tsv":
                file_bt = pd.read_csv(os.path.join(blobtools_dir, directory, file), sep="\t")
                # find column ending with "_cov" and rename to "coverage"
                cov_cols = [col for col in file_bt.columns if col.endswith("_cov")]
                if cov_cols:
                    file_bt.rename(columns={cov_cols[0]: "coverage"}, inplace=True)
                # add sample ID from directory name
                file_bt['ID'] = directory
                file_bt.drop("index", axis=1, inplace=True)
                list_blobtools.append(file_bt)
# concat list
dat_blobtools = pd.concat(list_blobtools, ignore_index=True)
# clean up dataframe
dat_blobtools.rename(columns={
    "identifiers" : "Contig", 
    "length" : "Length",
    "coverage" : "Coverage",
    "gc" : "GC content",
    "bestsumorder_superkingdom" : "Superkingdom",
    "bestsumorder_kingdom" : "Kingdom",
    "bestsumorder_phylum" : "Phylum",
    "bestsumorder_class" : "Class",
    "bestsumorder_order" : "Order",
    "bestsumorder_family" : "Family",
    "bestsumorder_species" : "Species"
}, inplace=True)

# join with sample names
dat_blobtools = pd.merge(samples, dat_blobtools, how="left", on="ID")
# add column to state if assembly present
dat_blobtools["Assembled sequence"] = dat_blobtools["Contig"].apply(lambda x: "False" if pd.isna(x) else "True")

dat_blobtools = dat_blobtools[["ID", "Assembled sequence", "Contig", "Length", "Coverage", "GC content", "Superkingdom", "Kingdom", "Phylum", "Class", "Order", "Family", "Species"]]
dat_blobtools


,ID,Assembled sequence,Contig,Length,Coverage,GC content,Superkingdom,Kingdom,Phylum,Class,Order,Family,Species
0,Adelpha_iphiclus,True,Adelpha_iphiclus_circular,15263.0,17.0734,0.1946,no-hit,Metazoa,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Adelpha iphiclus
1,Anartia_jatrophae_saturata,True,Anartia_jatrophae_saturata_contig0,15281.0,17.0647,0.1862,no-hit,Metazoa,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Yoma algina
2,Araschnia_levana,True,Araschnia_levana_circular,15207.0,17.0016,0.1835,no-hit,Metazoa,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Araschnia levana
3,Auzakia_danava,True,Auzakia_danava_contig0,6944.0,16.5379,0.2036,no-hit,Metazoa,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Auzakia danava
4,Auzakia_danava,True,Auzakia_danava_contig1,4869.0,17.5841,0.1818,no-hit,Metazoa,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Auzakia danava
5,Auzakia_danava,True,Auzakia_danava_contig2,3204.0,18.4320,0.1898,no-hit,Metazoa,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Auzakia danava
6,Baeotus_beotus,True,Baeotus_beotus_circular,15131.0,17.0449,0.1954,no-hit,Metazoa,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Baeotus beotus
7,Catacroptera_cloanthe,True,Catacroptera_cloanthe_contig0,13058.0,17.4831,0.2261,no-hit,Metazoa,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Catacroptera cloanthe
8,Chalinga_pratti,True,Chalinga_pratti_contig0,15244.0,17.1300,0.1848,no-hit,Metazoa,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Chalinga pratti
9,Diaethria_gabaza_eupepla,True,Diaethria_gabaza_eupepla_circular,15156.0,17.0368,0.1951,no-hit,Metazoa,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Ariadne ariadne


In [74]:
annotations_dir = "../../results/annotations/"

list_bed = []

# iterate throught annotations results files
for file in os.listdir(annotations_dir): 
    if os.path.isdir(os.path.join(annotations_dir, file)):
        for (root,dirs,files) in os.walk(os.path.join(annotations_dir, file), topdown=True):
            for f in files: 
                if f == "result.bed":
                    dat_ann = pd.read_csv(os.path.join(root, f), sep="\t", header=None, names=["Contig", "Start", "End", "Name", "Score", "Strand"])
                    # add sample name to dataframe
                    dat_ann["ID"] = file
                    # append to list
                    list_bed.append(dat_ann)
# concat list
dat_bed = pd.concat(list_bed, ignore_index=True)

# remove annotations for tRNAs and origin of replication
dat_bed = dat_bed[~dat_bed["Name"].str.startswith(("trn", "OH", "OL"))]

# join with sample names
# dat_bed = pd.merge(samples, dat_bed, how="left", on="ID")

# add column to state if assembly present
#dat_bed["Assembled sequence"] = dat_bed["Contig"].apply(lambda x: "False" if pd.isna(x) else "True")

dat_bed


,Contig,Start,End,Name,Score,Strand,ID
0,Anartia_jatrophae_saturata_contig0,4,1502,nad5_0,237982419.4,-,Anartia_jatrophae_saturata
2,Anartia_jatrophae_saturata_contig0,1586,2922,nad4,214655501.8,-,Anartia_jatrophae_saturata
3,Anartia_jatrophae_saturata_contig0,2921,3206,nad4l,4178913.3,-,Anartia_jatrophae_saturata
6,Anartia_jatrophae_saturata_contig0,3369,3867,nad6,3279670.2,+,Anartia_jatrophae_saturata
7,Anartia_jatrophae_saturata_contig0,3895,5047,cob,323602426.3,+,Anartia_jatrophae_saturata
...,...,...,...,...,...,...,...
921,Catacroptera_cloanthe_contig0,8669,9347,atp6,35353613.3,-,Catacroptera_cloanthe
922,Catacroptera_cloanthe_contig0,9340,9502,atp8,65220.0,-,Catacroptera_cloanthe
925,Catacroptera_cloanthe_contig0,9603,10317,cox2,107385243.0,-,Catacroptera_cloanthe
927,Catacroptera_cloanthe_contig0,10379,11915,cox1,566136198.5,-,Catacroptera_cloanthe


In [76]:
# get gene counts per sample
gene_counts = dat_bed.groupby(["ID"])[["Name"]].count().reset_index()

# get the numer of "cox1" annotations per sample
cox1_counts = dat_bed.groupby(["ID"])["Name"].apply(lambda x: (x == "cox1").sum())

# combine gene counts and cox1 counts into a single dataframe
dat_annotations = pd.merge(gene_counts, cox1_counts, how="outer", on="ID").rename(columns={"Name_x": "N annotations", "Name_y": "N cox1"})
dat_annotations

,ID,N annotations,N cox1
0,Adelpha_iphiclus,16,1
1,Anartia_jatrophae_saturata,17,1
2,Araschnia_levana,15,1
3,Auzakia_danava,18,1
4,Baeotus_beotus,15,1
5,Catacroptera_cloanthe,14,1
6,Chalinga_pratti,15,1
7,Diaethria_gabaza_eupepla,15,1
8,Doleschallia_melana,17,1
9,Eurema_blanda,15,1


In [77]:
# input directory for annotated sequences
extract_alignments_dir = "../../results/mafft_filtered/"

# functions
def read_fasta(filename):
    name, seq = None,''
    fasta = open(filename, 'r')
    for line in fasta:
        if line.startswith('>') and name == None:
            name = line.rstrip('\n').replace('>','')
        else:
            if line.startswith('>') and name != None:
                yield [name, seq]
                name = line.rstrip('\n').replace('>','')
                seq = ''
            else:
                seq = seq + line.rstrip('\n')
    yield [name, seq]
    fasta.close()

def format_name(name):
    name = name.split(';')[0]
    name = re.sub(r"_contig\d*$|","", name)
    name = re.sub(r"_circular|","", name)
    return name

# create empty dictionary to hold gene (keys) and samples (values)
#dict_genes = {}
list_genes = []


# list files in input dir
for file in os.listdir(extract_alignments_dir):
    # files ending with ".fasta"
    if file.endswith("fasta"):
        # get gene from file name
        gene = file.replace(".fasta","")
        # read fasta
        fasta = read_fasta(os.path.join(extract_alignments_dir, file))
        #iterate through sequences in fasta
        for name, seq in fasta:
            contig, start_stop, strand, name = name.split(";")
            start, stop = start_stop.split("-")
            # get sample name from contig
            sample = format_name(contig)
            list_genes.append([sample, contig, start, stop, strand, gene])
            
            ## if long assembly name found, reformat to sample only
            #if re.search("contig", name) or re.search("circular", name):
            #    name = format_name(name)
            ## add gene and name to dictionary
            #if dict_genes.get(gene) == None:
            #    dict_genes[gene] = [name]
            #else:
            #    dict_genes[gene].append(name)

dat_genes = pd.DataFrame(list_genes, columns=["ID", "Contig", "Start", "End", "Strand", "Name"])
dat_genes["Present_mafft_filtered"] = "True"
#dat_genes["Start"] = dat_genes["Start"].astype(float)
#dat_genes["End"] = dat_genes["End"].astype(float)
#dat_genes.drop("Start", axis=1, inplace=True)
#dat_genes.drop("End", axis=1, inplace=True)
dat_genes
#dict_genes

# get dict_genes sorted by key
#dict_genes_keys = list(dict_genes.keys())
#dict_genes_keys.sort()
#dict_genes_sorted = {i: dict_genes[i] for i in dict_genes_keys}

#dict_genes_sorted

,ID,Contig,Start,End,Strand,Name,Present_mafft_filtered
0,Anartia_jatrophae_saturata,Anartia_jatrophae_saturata_contig0,3896,5047,+,cob,True
1,Chalinga_pratti,Chalinga_pratti_contig0,9205,10359,-,cob,True
2,Protogoniomorpha_temora,Protogoniomorpha_temora_contig0,8938,10089,-,cob,True
3,Parasarpa_zayla,Parasarpa_zayla_circular,9469,10617,-,cob,True
4,Junonia_villida,Junonia_villida_contig3,269,1420,+,cob,True
...,...,...,...,...,...,...,...
365,Precis_pelarga,Precis_pelarga_circular,4749,5426,+,atp6,True
366,Salamis_cacta,Salamis_cacta_contig0,2970,3647,-,atp6,True
367,Doleschallia_melana,Doleschallia_melana_contig0,9348,10025,+,atp6,True
368,Litinga_cottini,Litinga_cottini_contig0,9139,9816,+,atp6,True


In [78]:
dat_bed

,Contig,Start,End,Name,Score,Strand,ID
0,Anartia_jatrophae_saturata_contig0,4,1502,nad5_0,237982419.4,-,Anartia_jatrophae_saturata
2,Anartia_jatrophae_saturata_contig0,1586,2922,nad4,214655501.8,-,Anartia_jatrophae_saturata
3,Anartia_jatrophae_saturata_contig0,2921,3206,nad4l,4178913.3,-,Anartia_jatrophae_saturata
6,Anartia_jatrophae_saturata_contig0,3369,3867,nad6,3279670.2,+,Anartia_jatrophae_saturata
7,Anartia_jatrophae_saturata_contig0,3895,5047,cob,323602426.3,+,Anartia_jatrophae_saturata
...,...,...,...,...,...,...,...
921,Catacroptera_cloanthe_contig0,8669,9347,atp6,35353613.3,-,Catacroptera_cloanthe
922,Catacroptera_cloanthe_contig0,9340,9502,atp8,65220.0,-,Catacroptera_cloanthe
925,Catacroptera_cloanthe_contig0,9603,10317,cox2,107385243.0,-,Catacroptera_cloanthe
927,Catacroptera_cloanthe_contig0,10379,11915,cox1,566136198.5,-,Catacroptera_cloanthe


In [79]:
def gene_present_in_dat_genes(dat_genes, id, contig, name):
    x = dat_genes [
        (dat_genes["ID"] == id) &
        (dat_genes["Contig"] == contig) &
        (dat_genes["Name"] == name)
        ]
    if x.shape[0] > 0:
        return True
    return False

dat_bed["Present"] = dat_bed.apply(lambda row: gene_present_in_dat_genes(dat_genes, row["ID"], row["Contig"], row["Name"]), axis=1)

dat_bed

,Contig,Start,End,Name,Score,Strand,ID,Present
0,Anartia_jatrophae_saturata_contig0,4,1502,nad5_0,237982419.4,-,Anartia_jatrophae_saturata,False
2,Anartia_jatrophae_saturata_contig0,1586,2922,nad4,214655501.8,-,Anartia_jatrophae_saturata,True
3,Anartia_jatrophae_saturata_contig0,2921,3206,nad4l,4178913.3,-,Anartia_jatrophae_saturata,True
6,Anartia_jatrophae_saturata_contig0,3369,3867,nad6,3279670.2,+,Anartia_jatrophae_saturata,True
7,Anartia_jatrophae_saturata_contig0,3895,5047,cob,323602426.3,+,Anartia_jatrophae_saturata,True
...,...,...,...,...,...,...,...,...
921,Catacroptera_cloanthe_contig0,8669,9347,atp6,35353613.3,-,Catacroptera_cloanthe,True
922,Catacroptera_cloanthe_contig0,9340,9502,atp8,65220.0,-,Catacroptera_cloanthe,True
925,Catacroptera_cloanthe_contig0,9603,10317,cox2,107385243.0,-,Catacroptera_cloanthe,True
927,Catacroptera_cloanthe_contig0,10379,11915,cox1,566136198.5,-,Catacroptera_cloanthe,True


In [80]:
dat_bed [ dat_bed["Present"] == False ]

,Contig,Start,End,Name,Score,Strand,ID,Present
0,Anartia_jatrophae_saturata_contig0,4,1502,nad5_0,2.379824e+08,-,Anartia_jatrophae_saturata,False
11,Anartia_jatrophae_saturata_contig0,6123,6860,rrnL_0,0.000000e+00,-,Anartia_jatrophae_saturata,False
12,Anartia_jatrophae_saturata_contig0,7146,7248,rrnL_1,3.800000e-03,-,Anartia_jatrophae_saturata,False
38,Anartia_jatrophae_saturata_contig0,15083,15275,nad5_1,9.939820e+04,-,Anartia_jatrophae_saturata,False
76,Protogoniomorpha_temora_contig0,1,607,cox3_0,8.055191e+07,-,Protogoniomorpha_temora,False
113,Protogoniomorpha_temora_contig0,15057,15333,cox3_1,2.041177e+07,-,Protogoniomorpha_temora,False
132,Parasarpa_zayla_circular,7257,7359,rrnL_1,1.500000e-03,+,Parasarpa_zayla,False
133,Parasarpa_zayla_circular,7647,8392,rrnL_0,0.000000e+00,+,Parasarpa_zayla,False
156,Junonia_villida_contig0,1951,2140,nad6,8.062231e+05,+,Junonia_villida,False
157,Junonia_villida_contig3,74,236,nad6,1.759792e+05,+,Junonia_villida,False


In [82]:
# get gene counts per sample
dat_genes.groupby(["ID"])[["Name"]].count().reset_index()

,ID,Name
0,Adelpha_iphiclus,15
1,Anartia_jatrophae_saturata,15
2,Araschnia_levana,15
3,Auzakia_danava,13
4,Baeotus_beotus,15
5,Catacroptera_cloanthe,14
6,Chalinga_pratti,15
7,Diaethria_gabaza_eupepla,15
8,Doleschallia_melana,15
9,Eurema_blanda,15
